<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/differential-privacy/blob/main/python/dp_auditorium/dp_auditorium/examples/run_mean_mechanism_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/differential-privacy/blob/main/python/dp_auditorium/dp_auditorium/examples/run_mean_mechanism_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

<br>
<br>

Or see this example in a [Python file](https://github.com/google/differential-privacy/blob/main/python/dp_auditorium/dp_auditorium/examples/run_mean_mechanism_example.py).

In [ ]:
#@title Install and import dp_auditorium and all necessary libraries.
!pip install google-vizier equinox
!git clone https://github.com/google/differential-privacy.git
import sys
sys.path.append('differential-privacy/python/dp_auditorium')

from dp_auditorium import interfaces
from dp_auditorium import privacy_test_runner
from dp_auditorium.generators import vizier_dataset_generator
from dp_auditorium.configs import dataset_generator_config
from dp_auditorium.configs import privacy_property
from dp_auditorium.configs import privacy_test_runner_config

import numpy as np
import dataclasses

In [ ]:
#@title Mean mechanism example

from typing import Callable
import time

# https://github.com/google/differential-privacy/blob/main/python/dp_auditorium/dp_auditorium/examples/run_mean_mechanism_example.py
def default_generator_factory(
    config: dataset_generator_config.VizierDatasetGeneratorConfig,
) -> vizier_dataset_generator.VizierScalarDataAddRemoveGenerator:
  return vizier_dataset_generator.VizierScalarDataAddRemoveGenerator(
      config=config
  )


def mean_mechanism_report(
    epsilon: float,
    delta: float,
    seed: int,
    generator_factory: Callable[
        [dataset_generator_config.VizierDatasetGeneratorConfig],
        vizier_dataset_generator.VizierScalarDataAddRemoveGenerator,
    ] = default_generator_factory,
) -> privacy_test_runner_config.PrivacyTestRunnerResults:
  """Runs the example code for a mean mechanism.

  Args:
    epsilon: standard DP parmaeter.
    delta: standard DP parameter.
    seed: seed to initialize the random number generator.
    generator_factory: factory to create a generator; to be replaced in tests

  Returns:
    The result of the example code as PrivacyTestRunnerResults.
  """
  rng = np.random.default_rng(seed=seed)
  tf.random.set_seed(seed)

  # Configuration for a non-private mean mechanism that uses the true number of
  # points to calculate the average and the scale of the noise.
  mech_config = mechanism_config.MeanMechanismConfig(
      epsilon=epsilon,
      delta=delta,
      use_noised_counts_for_calculating_mean=False,
      use_noised_counts_for_calculating_noise_scale=False,
      min_value=0.0,
      max_value=1.0,
  )
  # Initialize the mechanism.
  mechanism = mean.MeanMechanism(mech_config, rng)

  # Configuration for a Hockey-Stick property tester. Given arrays s1 and s2
  # with samples two distributions it will estimate the hockey-stick divergence
  # from the underlying distributions. It checks if the divergence is bounded by
  # delta.
  tester_config = property_tester_config.HockeyStickPropertyTesterConfig(
      training_config=hockey_stick_tester.make_default_hs_training_config(),
      approximate_dp=privacy_property.ApproximateDp(
          epsilon=epsilon,
          delta=delta,
      ),
  )
  # Initialize a classifier model for the Hockey-Stick property tester.
  # This classifier will learn to distinguish between samples of the mechanism
  # on adjacent datasets. Its accuracy level should be controlled by the privacy
  # guarantee.
  base_model = hockey_stick_tester.make_default_hs_base_model()
  # Initialize a property tester.
  property_tester = hockey_stick_tester.HockeyStickPropertyTester(
      config=tester_config,
      base_model=base_model,
  )

  # Configuration for dataset generator. It generates neighboring datasets under
  # the add/remove definition. Unique study name prevents using cached results
  # from previous runs.
  generator_config = dataset_generator_config.VizierDatasetGeneratorConfig(
      study_name=str(time.time()),
      study_owner="owner",
      num_vizier_parameters=2,
      data_type=dataset_generator_config.DataType.DATA_TYPE_FLOAT,
      min_value=-1.0,
      max_value=1.0,
      search_algorithm="RANDOM_SEARCH",
      metric_name="hockey_stick_divergence",
  )
  # Initialize the dataset generator.
  dataset_generator = generator_factory(generator_config)

  # Configuration for the test runner.
  # The test runner coordinates how the test is evaluated. It receives a
  # dataset generator, a property tester and a configuration (see base class for
  # details on these parameters), and runs privacy tests using the property
  # tester on datasets generated by the dataset generator.
  test_runner_config = privacy_test_runner_config.PrivacyTestRunnerConfig(
      property_tester=privacy_test_runner_config.PropertyTester.HOCKEY_STICK_TESTER,
      max_num_trials=10,
      failure_probability=0.05,
      num_samples=10_000,
      # Apply a hyperbolic tangent function to the output of the mechanism
      post_processing=privacy_test_runner_config.PostProcessing.TANH,
  )
  # Initialize the test runner.
  test_runner = privacy_test_runner.PrivacyTestRunner(
      config=test_runner_config,
      dataset_generator=dataset_generator,
      property_tester=property_tester,
  )

  return test_runner.test_privacy(mechanism, "non-private-mean-mechanism")


EPSILON = 1.0
DELTA = 1e-5
SEED = 1

# The results indicate whether a privacy violation was identified within the
# designated number of trials defined in the configuration. In the absence of a
# violation, a message is returned indicating that the limit of the number of
# trials has been reached. For reference, all computed divergences across all
# trials are also reported.
results = mean_mechanism_report(EPSILON, DELTA, SEED)
print(f" \nResults: \n{results}")